# JUPYTER FOR PYTHON PLOTS
____________________________

In [60]:
# standard
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import math
import os
import random
import ntpath
ntpath.realpath = ntpath.abspath
%matplotlib inline

In [34]:
markers = itertools.cycle(['o','s','v', ',', '+', '.', 'o', '*'])
colors = itertools.cycle(sns.color_palette())
kPSep = os.sep
directory_path = os.getcwd()
print(directory_path)
resultsFolder = ".."+kPSep + ".." + kPSep

D:\Uni\SEMESTERS\PRACE\CONDENSED_GROUP_CLOUD_UNI\DQMC\DQMC\Python\plots


### HUBBARD CLASS 

In [28]:
class HubbardDQMC:
    N = 1
    M = 1
    def __init__(self, M, M_0, U, mu, beta, lattice_type, Lx, Ly, Lz = 1, dim = 2):
        self.M = M
        self.M_0 = M_0
        self.dtau = beta/M
        self.lambd = math.acosh(math.exp((abs(U) * self.dtau) * 0.5));
        # physical
        self.beta = beta
        self.T = 1.0/beta
        self.U = U
        self.mu = mu
        # lattice
        self.lat_type = lattice_type
        self.dim = dim
        self.Lx = Lx
        self.Ly = Ly
        self.Lz = Lz
        self.N = Lx*Ly*Lz
        self.directory = "results" + kPSep 
        
    def getInfo(self):
        return "M=" + str(self.M) + ",M0=" + str(self.M_0) + \
		",dtau=" + str(self.dtau) + ",Lx=" + str(self.Lx) + \
		",Ly=" + str(self.Ly) + ",Lz=" + str(self.Lz) + \
		",beta=" + str(self.beta) + ",U=" + str(self.U) + \
		",mu=" + str(self.mu);
    
    def getDirectory(self):
        LxLyLz = "Lx=" + str(self.Lx) + ",Ly=" + str(self.Ly) + ",Lz=" + str(self.Lz)        
        return self.directory + self.lat_type + kPSep + str(self.dim) + "D" + kPSep + LxLyLz + kPSep

### TIME DISPLACED GREENS

In [62]:
def parseG_elems(line, nextline = ""):
    positions = "G(nx,ny,ti)"
    
    if line.lstrip().startswith(positions):
        tmp = nextline.split("\t")
        return ("r",int(tmp[1]),int(tmp[3]))
    else:
        tmp = line.split("\t")
        return (int(tmp[0]), float(tmp[1]), float(tmp[3]))

def readTimeDisplacedGreens(directory, M, number : str):
    pos_df = pd.DataFrame()
    pos_err_df =  pd.DataFrame()
    neg_df =  pd.DataFrame()
    neg_err_df =  pd.DataFrame()

    for filename in filter(lambda x: x.startswith(str(number)), os.listdir(directory)):
        print(directory + filename)
        with open(directory + filename, 'r') as f:
            # skip header
            for _ in range(33):
                next(f)
            # read lines
            for line in f:
                (dec,x,y) = parseG_elems(line, next(f))
                if dec == 'r':
                    tmp_col = (x,y)
                    tmp_arr = []
                    tmp_arr_err = []
                    for i in range(M):
                        (tau, val, err) = parseG_elems(next(f))
                        tmp_arr.append(val)
                        tmp_arr_err.append(err)

                    if "up" in filename:
                        pos_df[(tmp_col)] = np.array(tmp_arr)
                        pos_err_df[(tmp_col)] = np.array(tmp_arr_err)
                    else:
                        neg_df[(tmp_col)] = np.array(tmp_arr)
                        neg_err_df[(tmp_col)] = np.array(tmp_arr_err)
                
        f.close()
    return (pos_df, pos_err_df, neg_df, neg_err_df)

os.chdir( resultsFolder )
a = readTimeDisplacedGreens("results\\square\\2D\\Lx=8,Ly=8,Lz=1\\greens\\M=40,M0=8,dtau=0.05,Lx=8,Ly=8,Lz=1,beta=2.0,U=8.0,mu=0.0\\times\\", 40, "10-")
a[2]

### AVERAGE

In [ ]:
def averageTimeDisplacedGreens(directory, 

In [59]:
def plotTimeDisplacedGreens(directory, x, y):
    fig, ax = plt.subplots(4,2,figsize = (16,26))
    up = np.loadtxt(folder + "up.txt")
    down = np.loadtxt(folder + "down.txt")
    up_k = np.loadtxt(folder + "up_k.txt")
    down_k = np.loadtxt(folder + "down_k.txt")
    up_k = up_k * (2*np.pi) / Lx
    down_k = down_k *(2*np.pi) / Lx

    their = np.loadtxt(folder + "their.txt")
    their_k = np.loadtxt(folder + "their_k.txt")

    both = (down + up)*0.5
    both_k = (down_k + up_k)*0.5

    diff = np.abs(their - both)
    diff_k = np.abs(their_k - both_k)

    fig.suptitle("Comparison of time displaced Green's functions for $\mu$ = -4, number of bucket averages $N_A$ = 50", va = "center", y = 0.9)

    ax[0][0].plot(up, label = "$\sigma$ = ↑")
    ax[0][0].plot(down, label = "$\sigma = $↓")
    ax[0][1].plot(up_k, label = "$\sigma$ = ↑")
    ax[0][1].plot(down_k, label = "$\sigma = $↓")
    ax[1][0].plot(both)
    ax[1][1].plot(both_k)
    ax[2][0].plot(their)
    ax[2][1].plot(their_k)
    ax[3][0].plot(diff)
    ax[3][1].plot(diff_k)


    ax[0][0].legend()
    ax[0][1].legend()

    ax[0][0].set_xlabel("τ/dτ")
    ax[0][1].set_xlabel("τ/dτ")
    ax[1][0].set_xlabel("τ/dτ")
    ax[1][1].set_xlabel("τ/dτ")
    ax[2][0].set_xlabel("τ/dτ")
    ax[2][1].set_xlabel("τ/dτ")
    ax[3][0].set_xlabel("τ/dτ")
    ax[3][1].set_xlabel("τ/dτ")

    ax[0][0].set_ylabel("$G_σ$([0,0],τ/dτ)")
    ax[0][1].set_ylabel("$G_{kσ}$([0,0],τ/dτ)")
    ax[1][0].set_ylabel("$G_σ$([0,0],τ/dτ)")
    ax[1][1].set_ylabel("$G_{kσ}$([0,0],τ/dτ)")
    ax[2][0].set_ylabel("$G_σ$([0,0],τ/dτ)")
    ax[2][1].set_ylabel("$G_{kσ}$([0,0],τ/dτ)")
    ax[3][0].set_ylabel("$G_σ$([0,0],τ/dτ)")
    ax[3][1].set_ylabel("$G_{kσ}$([0,0],τ/dτ)")

    ax[0][0].set_title("G([0,0],τ) for ↑ and ↓ spin channels")
    ax[0][1].set_title("$G_k$([$k_x$=0,$k_y$=0],τ) for ↑ and ↓ spin channels")
    ax[1][0].set_title("Average G([0,0],τ) of both spin channels")
    ax[1][1].set_title("Average $G_k$([$k_x$=0,$k_y$=0],τ) of both spin channels")
    ax[2][0].set_title("G([0,0],τ) from US")
    ax[2][1].set_title("$G_k$([$k_x$=0,$k_y$=0],τ) from US")
    ax[3][0].set_title("Difference of US and my data : G([0,0],τ)")
    ax[3][1].set_title("Difference of US and my data : $G_k$([$k_x$=0,$k_y$=0],τ)")

    ax[3][0].set_yscale("log")
    ax[3][1].set_yscale("log")

    ax[0][0].set_xlim(0,39)
    ax[0][1].set_xlim(0,39)
    ax[1][0].set_xlim(0,39)
    ax[1][1].set_xlim(0,39)
    ax[2][0].set_xlim(0,39)
    ax[2][1].set_xlim(0,39)
    ax[3][0].set_xlim(0,39)
    ax[3][1].set_xlim(0,39)

    ax[0][0].set_ylim(-1,0)
    ax[0][1].set_ylim(-1,0)
    ax[1][0].set_ylim(-1,0)
    ax[1][1].set_ylim(-1,0)
    ax[2][0].set_ylim(-1,0)
    ax[2][1].set_ylim(-1,0)
    ax[3][0].set_ylim(1e-3,8e-2)
    ax[3][1].set_ylim(1e-2,5e-2)

    plt.savefig(folder+"greens_mu=-4,Na=50.pdf")